#### CRUD operations

Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

**TODO 1**: Take the dict created in the TODO 4 in chapter II and save it in the collection "CRUD_exercise".

**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

**TODO 3**: Insert the dict created in the TODO 6 Chapter II in the example collection.

**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

**TODO 5**: Change the documents where x = 4 to x = 1.

**TODO 6**: Find documents where author is not_mike and set author as real_mike.

**TODO 7**: Delete documents where author is real_mike.

#### Managing DB

**TODO 8**: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

**TODO 9**: create a random collection in a random db and put the new collection in the tutorial DB

**TODO 10**: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

#### Real world problems

**TODO 11**:  Use the oaipmh and api code get papers "econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "econ".

**TODO 12**: Do the same as TODO 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

**TODO 13**: Download a random image and store it in a collection.

**TODO 14**: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

**TODO 15**: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

**TODO 16**: Download a [sound sample](https://freesound.org/browse/). Try to store it in MongoDB 

**TODO 17**: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ?

In [ ]:
pip install pymongo

In [4]:
import pymongo
import tqdm

In [2]:
import pymongo
import numpy as np

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["Todo"]
CRUD_exercise = db["CRUD_exercise"]

In [ ]:

#1
doc = {"name": "John", "age": 25, "city": "New York"}
CRUD_exercise.insert_one(doc)

#2
CRUD_exercise.insert_many([{"x": 1}, {"x": 1}, {"x": 1}])
CRUD_exercise.delete_one({"x": 1})
CRUD_exercise.update_many({}, {"$inc": {"x": 3}})

#3
example = db["example"]
doc = {"author": "Mike", "text": "My first blog post!", "tags": ["mongodb", "python", "pymongo"]}
example.insert_one(doc)

#4
authors = CRUD_exercise.find({"authors": {"$exists": True}})
for author in authors:
    print(author)

#5
CRUD_exercise.update_many({"x": 4}, {"$set": {"x": 1}})

#6
CRUD_exercise.update_many({"author": "not_mike"}, {"$set": {"author": "real_mike"}})

#7
CRUD_exercise.delete_many({"author": "real_mike"})

#8
CRUD_exercise_benchmark = db["CRUD_exercise_benchmark"]
data = [{"id": i, "values": np.random.rand(10)} for i in range(0, 1000000, 2)]
CRUD_exercise_benchmark.insert_many(data)
CRUD_exercise_benchmark.create_index([("id", pymongo.ASCENDING)])
result_before = CRUD_exercise_benchmark.find({"id": 100}).explain()["executionStats"]
result_after = CRUD_exercise_benchmark.find({"id": 100}).explain()["executionStats"]

#9
random_db = client["Random"]
random_collection = random_db["Random_collection"]
random_collection.insert_one({"message": "This is a random collection"})
client.admin.command("copydb", fromdb="Random", todb="Todo")

#10
# Inner join returns only the matching documents from both collections whereas outer join returns all documents 
# from both collections, including non-matching documents as null. The query seen during the course was an inner join.

In [ ]:
#11
